<a href="https://colab.research.google.com/github/mimingucci/ML/blob/main/NaiveBayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Naive Bayes assumes the random variables within $x_n$ are independent conditional on the class of observation n. I.e. if $x_n \in \mathbb{R}^D
$ , Naive Bayes assumes<br/>
$p(x_n|Y_n) = p(x_{n1}|Y_n)\cdot p(x_{n2}|Y_n) \cdot ... \cdot p(x_{nD}|Y_n).$
<br/>
**Making Classifications**
<br/>
Consider a test observation $x_0$. For k=1,…,K
, we use Bayes’ rule to calculate<br/>
$\begin{split}
\begin{align*}
p(Y_0 = k|x_0) &\propto p(x_0|Y_0 = k)p(Y_0 = k)
\\
&= \hat{p}(x_0|Y_0 = k)\hat{\pi}_k,
\end{align*}
\end{split}
$  <br/>
where $\hat{p}$ gives the estimated density of $x_0$ conditional on $Y_0$. We then predict $Y_0$=k
 for whichever value k
 maximizes the above expression.

In [ ]:
class NaiveBayes:

    #Fit Model

    def _estimate_class_parameters(self, X_k):

        class_parameters = []

        for d in range(self.D):
            X_kd = X_k[:,d] # only the dth column and the kth class

            if self.distributions[d] == 'normal':
                mu = np.mean(X_kd)
                sigma2 = np.var(X_kd)
                class_parameters.append([mu, sigma2])

            if self.distributions[d] == 'bernoulli':
                p = np.mean(X_kd)
                class_parameters.append(p)

            if self.distributions[d] == 'poisson':
                lam = np.mean(X_kd)
                class_parameters.append(p)

        return class_parameters

    def fit(self, X, y, distributions = None):

        #Record info
        self.N, self.D = X.shape
        self.X = X
        self.y = y
        if distributions is None:
            distributions = ['normal' for i in range(len(y))]
        self.distributions = distributions


        #Get prior probabilities
        self.unique_y, unique_y_counts = np.unique(self.y, return_counts = True) # returns unique y and counts
        self.pi_ks = unique_y_counts/self.N


        #Estimate parameters
        self.parameters = []
        for i, k in enumerate(self.unique_y):
            X_k = self.X[self.y == k]
            self.parameters.append(self._estimate_class_parameters(X_k))


    #Make Classifications

    def _get_class_probability(self, x_n, j):

        class_parameters = self.parameters[j] # j is index of kth class
        class_probability = 1

        for d in range(self.D):
            x_nd = x_n[d] # just the dth variable in observation x_n

            if self.distributions[d] == 'normal':
                mu, sigma2 = class_parameters[d]
                class_probability *= sigma2**(-1/2)*np.exp(-(x_nd - mu)**2/sigma2)

            if self.distributions[d] == 'bernoulli':
                p = class_parameters[d]
                class_probability *= (p**x_nd)*(1-p)**(1-x_nd)

            if self.distributions[d] == 'poisson':
                lam = class_parameters[d]
                class_probability *= np.exp(-lam)*lam**x_nd

        return class_probability

    def classify(self, X_test):

        y_n = np.empty(len(X_test))
        for i, x_n in enumerate(X_test): # loop through test observations

            x_n = x_n.reshape(-1, 1)
            p_ks = np.empty(len(self.unique_y))

            for j, k in enumerate(self.unique_y): # loop through classes

                p_x_given_y = self._get_class_probability(x_n, j)
                p_y_given_x = self.pi_ks[j]*p_x_given_y # bayes' rule

                p_ks[j] = p_y_given_x

            y_n[i] = self.unique_y[np.argmax(p_ks)]

        return y_n
